In [2]:
import sys
sys.path.append('/home/jp/Documents/GitProjects/faas-sim')
from typing import List

import matplotlib.pyplot as plt
from pandas import DataFrame
import numpy as np
import pickle
import datetime as dt
import matplotlib.dates as md
from matplotlib import ticker

from ext.jjnp21.automator.experiment import Result



def load_results(path) -> List[Result]:
    with open(path, 'rb') as f:
        results = pickle.load(f)
        return results

['/home/jp/Documents/GitProjects/faas-sim/ext/jjnp21/experiments/osmotic',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/home/jp/Documents/GitProjects/old-faasim-for-eval/faas-sim/venv/lib/python3.7/site-packages',
 '/home/jp/Documents/GitProjects/old-faasim-for-eval/faas-sim/venv/lib/python3.7/site-packages/IPython/extensions',
 '/home/jp/.ipython']

In [10]:
# importing of raw data
path = '/home/jp/Documents/tmp/'
dumps = [
    ('osmotic_basic_fixed_fx_scaling.dump', 'fixed_scaling'),
#     ('osmotic_basic_openfaas_scaling.dump', 'openfaas_scaling_basic'),
#     ('test.dump', 'test')
]
results = {}
for filename, key in dumps:
    results[key] = load_results(f'{path}/{filename}')

In [17]:
list(map(lambda x: x.experiment.name, results['fixed_scaling']))

['25rps_2000s_city_rr_baseline',
 '25rps_2000s_city_lrt_baseline',
 '25rps_2000s_city_osmotic',
 '50rps_2000s_city_rr_baseline',
 '50rps_2000s_city_lrt_baseline',
 '50rps_2000s_city_osmotic',
 '75rps_2000s_city_lrt_baseline',
 '75rps_2000s_city_osmotic',
 '25rps_2000s_nation_osmotic',
 '25rps_2000s_nation_lrt_baseline',
 '75rps_2000s_city_rr_baseline',
 '25rps_2000s_nation_rr_baseline',
 '50rps_2000s_nation_osmotic',
 '50rps_2000s_nation_lrt_baseline',
 '75rps_2000s_nation_lrt_baseline',
 '25rps_2000s_global_lrt_baseline',
 '25rps_2000s_global_osmotic',
 '75rps_2000s_nation_osmotic',
 '75rps_2000s_global_osmotic',
 '50rps_2000s_global_osmotic',
 '75rps_2000s_nation_rr_baseline',
 '50rps_2000s_nation_rr_baseline',
 '50rps_2000s_global_lrt_baseline',
 '75rps_2000s_global_lrt_baseline',
 '75rps_2000s_global_rr_baseline',
 '25rps_2000s_global_rr_baseline']

In [22]:
r.scale
# r.experiment.name

,value,function_name
time,,
2022-02-01 23:19:42.132444,5,resnet50-inference
2022-02-01 23:19:42.132444,5,resnet50-training
2022-02-01 23:19:42.132444,5,mobilenet-inference
2022-02-01 23:19:42.132444,5,speech-inference
2022-02-01 23:19:42.132444,5,resnet50-preprocessing
2022-02-01 23:20:12.132444,15,resnet50-training
2022-02-01 23:20:12.132444,15,resnet50-preprocessing
2022-02-01 23:20:12.132444,32,resnet50-inference
2022-02-01 23:20:12.132444,32,mobilenet-inference
